In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/msambare/fer2013/versions/1


In [ ]:
dataset_path = "/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1"

classes = ['angry' , 'disgust' , 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torch import nn, optim
from torch.utils.data import random_split
import os
from PIL import Image

In [ ]:
class ExpressionDataset(Dataset):
    def __init__(self, root_dir, classes, transform=None):
        self.root_dir = root_dir
        self.classes = classes
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label, class_name in enumerate(classes):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):  # Loop through the images inside the class folder
                img_path = os.path.join(class_dir, img_name)  # Full path to the image
                self.image_paths.append(img_path)  # Add the image path to the list
                self.labels.append(label)

    def __len__(self):
        """Return the total number of samples"""
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize(256),  # Resize the image to 256x256 (after augmentations)
    transforms.CenterCrop(224),  # Crop to 224x224
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(  # Normalize with ImageNet mean and std
        mean=[0.485, 0.456, 0.406],  # ImageNet mean
        std=[0.229, 0.224, 0.225]  # ImageNet std
    )
])

train_root = os.path.join(dataset_path, 'train')
test_root = os.path.join(dataset_path, 'test')

train_dataset = ExpressionDataset(train_root, classes, transform=transform)
test_dataset = ExpressionDataset(test_root, classes, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:

model = models.resnet50(pretrained=True)
model.fc = nn.Sequential(
    nn.Dropout(0.3),  # Dropout layer with 30% probability
    nn.Linear(model.fc.in_features, 7)  # Final classification layer for 7 classes
)

# Move the model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 123MB/s]


In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=1e-4)


# Training function
def train_model(model, train_loader, criterion, optimizer, device, num_epochs):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            # Move data to the selected device (GPU or CPU)
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Calculate statistics for logging
            running_loss += loss.item()

            _, predicted = outputs.max(1)  # Get the class with the highest score
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = correct / total * 100
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

    print("Training complete.")


In [ ]:
import torch
from torch.nn.functional import softmax

def test_model(model, test_loader, device):
    """
    Evaluate the model on the test dataset.

    Parameters:
        model (torch.nn.Module): The trained PyTorch model.
        test_loader (DataLoader): DataLoader for the test dataset.
        device (torch.device): The device to perform computations on (CPU or GPU).

    Returns:
        float: The accuracy of the model on the test dataset.
    """
    model.to(device)
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for inference
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_model(model, test_loader, device, cat_names):
    """
    Evaluate the model on the test data and print classification report and confusion matrix.

    Parameters:
    - model: The trained model
    - test_loader: The DataLoader for the test dataset
    - device: The device (CPU or GPU) on which the model is running
    - cat_names: List of category names (e.g., ['angry', 'happy', ...])
    """
    all_labels = []
    all_preds = []

    model.eval()  # Set the model to evaluation mode

    # No need to compute gradients during evaluation
    with torch.no_grad():
        for images, labels in test_loader:
            # Move the images and labels to the same device as the model (GPU or CPU)
            images, labels = images.to(device), labels.to(device)

            # Forward pass: Get model outputs
            outputs = model(images)

            # Get the predicted class by taking the index with the highest probability
            _, predicted = torch.max(outputs, 1)

            # Collect the true labels and predicted labels for later evaluation
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    # Print classification report and confusion matrix
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=classes))

    print("\nConfusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))



In [ ]:
# Function to save the model
def save_model(model, path='expression_model.pth'):
    torch.save(model.state_dict(), path)
    print(f'Model saved to {path}')


In [ ]:
# Train the model
num_epochs = 10
train_model(model, train_loader, criterion, optimizer, device, num_epochs)

# Test the model
test_model(model, test_loader, device)

evaluate_model(model, test_loader, device, classes)

# Save the model after training
save_model(model)
